## The softmax function


The softmax function can be written as follows, where $i$ indexes an instance in a dataset and there are $K$ output classes. 

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})} $

In this notebook we will learn about the softmax function by implementing it in Python code. This will also help us learn some very fundamental aspects of implementing machine learning algorithms. Concretely we will do the following: 
   - First, we will implement the softmax function using vanilla Python code.
   - Second, we will experiment with implementing the softmax function using vectorized code, and see that it will be much faster.
   - Finally, we will also implement a numerically stable and vectorized version of the softmax. This means that we will code a version of the sofmax function that will be able to handle really big or small inputs (which is common in ML).

### Step 1: Thinking through the dimensions of our data

Say we have 2 features and 3 classes. What is the dimensionality of $x_i$? What is the dimensionality of our scores vector? 

In [2]:
import numpy as np

K = 3   # let K be the number of classes
J = 2   # let J be the number of features

W = np.random.rand(K, J)  

x_i = None

scores = None  # scores should be a 1D vector of raw scores, one for each class

### Step 2: Plotting the exp function

- Plot the exp function from -3 to 3. 

- What will happen to the range as the domain gets very big or small?

In [3]:
import pandas as pd
import altair as alt

x = np.linspace(-3, 3, 50)
y = None

### Step 3: Naive softmax

In [22]:
def naive_sofmax(scores):
    '''
    Write a "naive" version of softmax that does not use vectorized operations
    This will be *way* slower than the numpy version, but that is OK
    We are just building intuition before jumping to the vectorized version
    When you don't know how to code something in ML, it can be good to start with a 
    simple version using loops before skipping ahead to the vectorized version
    '''
    numerator = None
    denominator = None

### Step 4: Vectorized code in general

Vectorized operations are much faster. A vectorized operation uses linear algebra packages like numpy to do computation very quickly using highly optimized code. Some implementations may also run vector operations in parallel.

Let's experiment with vectorized versions of a simple function summing a list of numbers, before moving on to a vectorized softmax.

In [5]:
import math 

def L2norm_no_vector(x):
    '''
    Your code here should not use vector operations
    
    input: a vector
    output, L2 norm of x, \sqrt{\Sigma x_i^2}
    '''
    return 42
        
def L2norm_with_vectors(x):
    '''
    Your code here should use vector operations

    input: a vector
    output, L2 norm of x, \sqrt{\Sigma x_i^2}
    '''
    return 42
        

Before going on, let's confirm that both implementations return the same outputs.

In [6]:
L2norm_no_vector([2,3,4]),  L2norm_with_vectors(np.asarray([2,3,4]))

(42, 42)

Let's test how long it takes our code to run as the list grows. 

In [7]:
from timeit import default_timer as timer

results = []

for n in range(1, 10000, 1000):
    runs = []
    for i in range(25): # take an average of 25 runs
        start = timer()
        L2norm_no_vector(np.random.rand(n))
        end = timer()
        runs.append(end - start)
    mean = np.mean(runs)
    results.append({"N": n, "f": 'L2norm_no_vector', "time": mean})
    
for n in range(1, 10000, 1000):
    runs = []
    for i in range(25): # take an average of 25 runs
        start = timer()
        L2norm_with_vectors(np.random.rand(n))
        end = timer()
        runs.append(end - start)
    mean = np.mean(runs)
    results.append({"N": n, "f": 'L2norm_with_vectors', "time": mean})
 
results = pd.DataFrame(results)
alt.Chart(results).mark_line().encode(
    x="N",
    y="time",
    color="f"
)

### Step 5: Vectorized softmax

Now we are ready to take our first crack at a vectorized softmax function. Remember the equation for the softmax is as follows:

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})} $


In [8]:
def softmax(scores):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of probabilities that sums to 1
    '''
    return 42

softmax([1, 1])

42

#### Softmax questions

1. What happens when the scores are the same?
2. What happens when one score is bigger than the others
3. Why do you think this function might be called a "soft" max?
4. What happens if you pass in the scores `[10000, 1]`
4. What happens if you pass in the scores `[0, 1]`

### Step 6: vectorized and numerically stable softmax 

Recall our softmax function

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})}$


We can always divide the numerator and denominator of a fraction by $c$. It will affect the value of the numerator and denominator but not their ratio. Hence the probabilities will remain unchanged.

In [1]:
3/9 == (3/2)/(9/2)

True

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k}) / e^c}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})/ e^c}$

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k}) exp(-c)}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}) exp(-c)}$

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k} - c) }{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime} } -c ) } = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})}$


In [67]:
def softmax_c(scores, c):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of probabilities that sums to 1
    '''
    return np.exp(scores - c)/np.sum(np.exp(scores - c))

# lets verify that all the math above is correct ... always a good idea ... 
softmax(np.asarray([1,2])),  softmax_c(np.asarray([1,2]), 2)

(array([0.26894142, 0.73105858]), array([0.26894142, 0.73105858]))

### Now let's take the log of the softmax

Here is our equation from above:

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k} - c)}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}  - c)}$

Now let's take the log:

$ log p(y_i = k | x_i) = \text{score}_{i, k} - c  - log {\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}  - c)}$

Note that we can use any $c$ we want in the above equation, so let's set $c$ to the largest item in our scores vector.

In [12]:

def log_softmax(scores):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of log probabilities. When expoentitated, these probabilities should sum to 1
    '''
    c = np.max(scores) # c = None # set c to the max of scores
    logsumexp = np.log(np.sum(np.exp(scores - c)))
    return scores - c - logsumexp



#### Tests for numerical stability and correctness

Test for numerical stability and correctness with the following code. What do you notice is different about your log softmax?

In [ ]:
print(softmax([10000, 1]))
print(np.exp(log_softmax([10000, 1])))
print(np.exp(log_softmax([5, 5])))